Copyright (c) Microsoft Corporation. All rights reserved. 

Licensed under the MIT License.

# Tutorial: How to train and deploy your model in Azure Machine Learning in 10 minutes

In this tutorial, you learn how to quickly get started with Azure Machine Learning. You will train an image classification model using the [MNIST](https://azure.microsoft.com/services/open-datasets/catalog/mnist/) dataset.

You will learn how to:

> * Download a dataset and look at the data
> * Train an image classification model and log metrics
> * Deploy the model

## Import Data

Before you train a model, you need to understand the data that you are using to train it. In this section you learn how to:

* Download the MNIST dataset
* Display some sample images

### Download the MNIST dataset

Use Azure Open Datasets to get the raw MNIST data files. [Azure Open Datasets](https://docs.microsoft.com/azure/open-datasets/overview-what-are-open-datasets) are curated public datasets that you can use to add scenario-specific features to machine learning solutions for more accurate models. Each dataset has a corrseponding class, `MNIST` in this case, to retrieve the data in different ways.

Follow this [how-to](https://aka.ms/azureml/howto/createdatasets) if you want to learn more about Datasets and how to use them.

<font color='red'>To evaluate: alternative to downloading as filedataset is to use MNIST.get_tabular_dataset(), which will allow to drop the utils package. Code to be found [here](https://azure.microsoft.com/en-us/services/open-datasets/catalog/mnist/)</font>

In [ ]:
from azureml.core import Dataset
from azureml.opendatasets import MNIST

data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

mnist_file_dataset = MNIST.get_file_dataset()
mnist_file_dataset.download(data_folder, overwrite=True)

mnist_file_dataset = mnist_file_dataset.register(workspace=ws,
                                                 name='mnist_opendataset',
                                                 description='training and test dataset',
                                                 create_new_version=True)

### Take a look at the data
Load the compressed files into `numpy` arrays. Then use `matplotlib` to plot 30 random images from the dataset with their labels above them. 

Note this step requires a `load_data` function that's included in an `utils.py` file. This file is placed in the same folder as this notebook. The `load_data` function simply parses the compressed files into numpy arrays.

In [ ]:
from utils import load_data
import glob


# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
X_train = load_data(glob.glob(os.path.join(data_folder,"**/train-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
X_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-images-idx3-ubyte.gz"), recursive=True)[0], False) / 255.0
y_train = load_data(glob.glob(os.path.join(data_folder,"**/train-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)
y_test = load_data(glob.glob(os.path.join(data_folder,"**/t10k-labels-idx1-ubyte.gz"), recursive=True)[0], True).reshape(-1)


# now let's show some randomly chosen images from the traininng set.
count = 0
sample_size = 30
plt.figure(figsize = (16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline('')
    plt.axvline('')
    plt.text(x=10, y=-10, s=y_train[i], fontsize=18)
    plt.imshow(X_train[i].reshape(28, 28), cmap=plt.cm.Greys)
plt.show()

## Train model and log metrics

The model is trained using the code below. You will create a new Experiment in your workspace, and your training runs and metrics will be registered there so that this information is available after you've finished training your model.

You will be using the [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) classifier from the [SciKit Learn framework](https://scikit-learn.org/) to classify the data.

* **Note: The model training take around 1 minute to complete.**

In [ ]:
# create experiment and start logging to a new run in the experiment
from azureml.core import Experiment
from azureml.core import Workspace

ws = Workspace.from_config()

exp = Experiment(workspace=ws, name='sklearn-mnist')
run = exp.start_logging(snapshot_directory=None)

In [ ]:
# create the model
import numpy as np
from sklearn.linear_model import LogisticRegression 

reg = 0.5
clf = LogisticRegression(C=1.0/reg, solver="liblinear", multi_class="auto", random_state=42)
clf.fit(X_train, y_train)

#make predictions using the test set and calculate the accuracy
y_hat = clf.predict(X_test) 

# calculate accuracy on the prediction
acc = np.average(y_hat == y_test)
print('Accuracy is', acc)

run.log('regularization rate', np.float(reg))
run.log('accuracy', np.float(acc))

## Version control your models with the Model Registry

You can use model registration to store and version your models in your workspace. Registered models are identified by name and version. Each time you register a model with the same name as an existing one, the registry increments the version. Azure Machine Learning supports any model that can be loaded through Python 3.

The code below does:

1. Saves the model to disk
1. Uploads the model file to the run 
1. Registers the uploaded model file
1. Transitions the run to a completed state

In [ ]:
import joblib
from azureml.core.model import Model

path = 'sklearn_mnist_model.pkl'
joblib.dump(value=clf, filename=path)

run.upload_file(name=path, path_or_stream=path)

model = run.register_model(model_name='sklearn_mnist_model', 
                                model_path=path,
                                description='Mnist handwriting recognition')

run.complete()

### View model in the model registry 

You can see the stored model by navigating to **Models** in the left-hand menu bar of Azure Machine Learning Studio. Click on the **sklearn_mnist_model** and you can see the details of the model like the experiement run id that created the model.

To evaluate: move this section to the end to prevent user from losing context? 

## Deploy the model

The next cells deploys the model to an Azure Container Instance so that you can score data in real-time (Azure Machine Learning also provides mechanisms to do batch scoring). A real-time endpoint allows application developers to integrate machine learning into their apps.

* **Note: The deployment takes around 3 minutes to complete.**

In [ ]:
#create environment for the deploy
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment('tutorial-env')
cd = CondaDependencies.create(pip_packages=['azureml-dataset-runtime[pandas,fuse]', 'azureml-defaults'], conda_packages = ['scikit-learn==0.22.1'])

env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace = ws)

#create config file
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "MNIST",  "method" : "sklearn"}, 
                                               description='Predict MNIST with sklearn')

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, 'sklearn_mnist')


myenv = Environment.get(workspace=ws, name="tutorial-env", version="1")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service_name = 'sklearn-mnist-svc-' + str(uuid.uuid4())[:4]
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aciconfig)

service.wait_for_deployment(show_output=True)

The [*scoring script*](score.py) file referenced in the code above can be found in the same folder as this notebook, and has two functions:

1. an `init` function that executes once when the service starts - in this function you normally get the model from the registry and set global variables
1. a `run(data)` function that executes each time a call is made to the service. In this function, you normally deserialize the json, run a prediction and output the predicted result.


## Test the model service

You can test the model by sending a raw HTTP request to test the web service. 

In [ ]:
# scoring web service HTTP endpoint
print(service.scoring_uri)

In [ ]:
#send raw HTTP request to test the web service.
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test)-1)
input_data = "{\"data\": [" + str(list(X_test[random_index])) + "]}"

headers = {'Content-Type':'application/json'}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)} 

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
#print("input data:", input_data)
print("label:", y_test[random_index])
print("prediction:", resp.text)

### View the results of your training

When you're finished with an experiment run you can always return to view the results of your model training here in the Azure Machine Learning studio:

1. Select **Experiments** (left-hand menu)
1. Select **sklearn-mnist**
1. Select **Run 1**
1. Select the **Metrics** Tab

The metrics tab will display the parameter values that were logged to the run.

### View the model in the model registry

You can see the stored model by navigating to **Models** in the left-hand menu bar. Click on the **sklearn_mnist_model** and you can see the details of the model like the experiement run id that created the model.

## Control cost and clean up resources

To clean up the resources after this quickstart, first you delete the Model service using:

In [ ]:
#if you want to keep workspace and only delete endpoint (it will incur cost while running)
service.delete()

If you want to control cost you can also stop the compute instance this notebook is running on by following these steps:

1. Go to **Compute** in the left-hand menu of the Azure Machine Learning studio
1. Select your compute instance
1. Select **Stop**


**Important: The resources you created can be used as prerequisites to other Azure Machine Learning tutorials and how-to articles.** If you don't plan to use the resources you created, delete them, so you don't incur any charges:

1. In the Azure portal, select **Resource groups** on the far left.
1. From the list, select the resource group you created.
1. Select **Delete resource group**.
1. Enter the resource group name. Then select **Delete**.

You can also keep the resource group but delete a single workspace. Display the workspace properties and select **Delete**.

## Next Steps

In this tutorial, you have seen how to run your machine learning code in Azure Machine Learning. 

It is often the case that once you have your machine learning code working in a development environment that you want to productionize this by using the Python SDK and run the code as a **_job_** - ideally on a schedule or trigger (for example, arrival of new data). To this end, we recommend that you follow the next tutorial in this series, [**Getting started with the Python SDK**](GettingStartedWithPythonSDK.ipynb). This 3-part tutorial is focused on running jobs-based machine learning code in the cloud.